# Calcudoku Solver

TODO:
 * Fix indexing with board to be consistent
 * Fix type annotations
 * Add more comments
 * Refactor variable names and lint
 * Finish working on optimizing the exhaustive search

The purpose of this project is to create a program which be able to solve a Calcudoku puzzle with $n < 12$ in a reasonable amount of time ( < 1 minute ).

## What's Calcudoku?

A Calcudoku puzzle is a puzzle played on a n x n grid. The grid is seperated into regions outlined by a dark lines. Each region is labeled with a resulting number and a subset of region are labeled with an operation. If a region is not labeled with an operation, then the number for that square is must be the same as the resulting number. The goal of the game is to fill each square on the grid with a number such that:

1. Each number is only used once in each row/column.

2. Only numbers between [1, n] are used

3. If a section has an operation associated with it, the numbers in a section must produce the section's resulting number when seperated by the section's operation.


### Example

This is what a simple 4x4 puzzle might look like before any numbers have been entered.


<img src='./imgs/ex_calc_1.png' style='height: 250px'>


Let's solve this one by hand to get an idea of how the game works. First you would need to fill in all the squares with no operation, since the value at those squares are known.


<img src='./imgs/ex_calc_2.png' style='height: 250px'>


Now. let's look at the bottom grid labeled `6+`. There are only two possible sets of numbers that add up to 6 between `[1,4]`: `3 + 3` or `2 + 4`. However the bottom row cannot have two 3's so it must be `2 + 4`. And since we already have a two in the second column, it must be ordered 2 then 4. So our puzzle becomes:


<img src='./imgs/ex_calc_3.png' style='height: 250px'>


Now since we know every number except one in the first and second column we can deduce the value of those to be 4 and 1 respectively. And if we know those values, then we also know the values bottom right must be 2 and 3.


<img src='./imgs/ex_calc_4.png' style='height: 250px'>


Now we can tackle the square near the top right labeled `6+`. Similar to before we know it has to be `4+2` and since there is already a 2 in the first row, then it must be ordered 4 then 2 (reading from top to bottom). Then we can just fill in the last two values for the top two rows without referencing the section labels.

<img src='./imgs/ex_calc_5.png' style='height: 250px'>


Now we're done! So let's get to work on an algorithm that'll solve these for us!

## Installing Necessary Packages

In [33]:
# from IPython.display import clear_output

# %pip install ipycanvas # For displaying the squares
# %pip install beautifulsoup4
# %pip install matplotlib
# # !jupyter labextension install @jupyter-widgets/jupyterlab-manager ipycanvas # To allow jupyter drawing
# # This may take a minute and you may need to restart the notebook after

# clear_output()

## Reading, Writing, and Storing the Calcudoku

Before we can start to design an algoritm for Calcudoku we need a way to load a puzzle, manipulate it, and display it on the screen. For simplicity's sake right now we can simply store the puzzle as a 2D array with a dictionary storing the groupings. We can then display the puzzle using ipycanvas.

In [34]:
from typing import List, Tuple, Set, Dict, Union
from ipycanvas import Canvas
from datetime import datetime
from functools import reduce


def multiply(*args):
    return reduce(lambda a,b: a*b, args)

def add(*args):
    return reduce(lambda a,b: a+b, args)

def divide(*args):
    return reduce(lambda a,b: a/b, args)

def subtract(*args):
    return reduce(lambda a,b: a-b, args)
    
class Board:
    DISPLAY_SQUARE_WIDTH = 100
    DISPLAY_SQUARE_HEIGHT = 100
    GROUP_LABEL_OFFSET = (10, 20)
    SOLUTION_OFFSET = (20, 45)
    MAX_TEXT_WIDTH = 11
    LINE_BREAK_WIDTH = 20
    GROUP_LABEL_FONT = '15px sans-serif'
    SOLUTION_FONT = '13px sans-serif'
    
    def __init__(self, n: int, calc_squares: List[dict]):
        self.possible_values = [[set(range(1, n+1)) for _ in range(n)] for _ in range(n)]
        self.calc_squares = calc_squares
        self.n = n

    def __getitem__(self, item: Union[int, Tuple[int, int]]):
        if type(item) == tuple:
            return self.possible_values[item[1]][item[0]]
        elif type(item) == int:
            return [self.possible_values[j][item] for j in range(self.n)]

    def __setitem__(self, key: Tuple[int, int], value: Set[int]):
        self.possible_values[key[1]][key[0]] = value
        
    def __eq__(self, other: object):
        if len(other.calc_squares) != len(self.calc_squares) or other.n != self.n:
            return False
        
        for calc_square in self.calc_squares:
            
            # Find the same calc_square in other
            matching_calc_squares = \
                [other_group for other_group in other.calc_squares \
                 if other_group['result'] == calc_square['result'] and \
                 other_group['operation'] == calc_square['operation']]
                
            
            if len(matching_calc_squares) != 1:
                return False
            
            matching_calc_square = matching_calc_squares.pop()
        
            # Make sure the other calc_square has the same positions
            for position in calc_square['positions']:
                if position not in matching_calc_square['positions']:
                    return False
        return True        
    
    def __len__(self):
        return self.n
        
    # Finds the top-leftmost point from a list of points
    @staticmethod
    def _top_left_most(points: List[Tuple[int, int]]) -> Tuple[int, int]:
        left_most = min(map(lambda a: a[0], points))
        return sorted(filter(lambda a: a[0] ==  left_most, points), key=lambda a: a[1])[0]
    
    # Adds a list of tuples
    @staticmethod
    def _add_tuples(*args: List[Tuple]) -> Tuple:
        return tuple(map(sum, zip(*args)))
    
    # Draws a line on a canvas from point start to point stop
    @staticmethod
    def _draw_line(canvas: Canvas, start: Tuple[int, int], stop: Tuple[int, int]) -> None:
        canvas.begin_path()
        canvas.move_to(*start)
        canvas.line_to(*stop)
        canvas.stroke()
    
    # For display purposes converts a grid coordinate e.g. (1, 1) to pixel 
    # coordinates e.g. (540, 540)
    def _grid_to_coords(self, x: int, y: int) -> Tuple[int, int]:
        return x*self.DISPLAY_SQUARE_WIDTH, y*self.DISPLAY_SQUARE_HEIGHT

    
    # Calculates where dark lines should be drawn in order to seperate
    # the math group math_group from everything else
    def _calculate_dark_lines(self, math_group: Dict[str, object]) -> List[Tuple[int, int]]:
        d = []

        for position in math_group['positions']:            
            left_position, right_position, top_position, bottom_position = None, None, None, None
            if position[0] > 0:
                left_position = {
                    'check': (-1, 0),
                    'p1': (0, 0),
                    'p2': (0, 1)
                }
            if position[0] < (self.n - 1):
                right_position = {
                    'check': (1, 0),
                    'p1': (1, 0),
                    'p2': (1, 1)
                }
            if position[1] > 0:
                top_position = {
                    'check': (0, -1),
                    'p1': (0, 0),
                    'p2': (1, 0)
                }
            if position[1] < (self.n - 1):
                bottom_position = {
                    'check': (0, 1),
                    'p1': (0, 1),
                    'p2': (1, 1)
                }
            count = 0
            for offset_position in (left_position, right_position, top_position, bottom_position):
                if offset_position:
                    check_pos = self._add_tuples(offset_position['check'], position)
                    if check_pos not in math_group['positions']:
                        count += 1
                        line_begin = self._add_tuples(offset_position['p1'], position)
                        line_end = self._add_tuples(offset_position['p2'], position)
                        d.append((self._grid_to_coords(*line_begin), 
                                  self._grid_to_coords(*line_end)))

        return d

    # Intelligently breaks a comma-seperated list apart so that single lines 
    # should not be MUCH wider than max_width
    @staticmethod
    def _break_at(string, max_width):
        split_string = string.split(',')
        
        # Add the commas back
        split_string = [sstring + ',' if ind != len(split_string) - 1 else sstring for ind, sstring in enumerate(split_string)]
        
        curr_line = 0
        for ind, sstring in enumerate(split_string):
            curr_line += len(sstring)
            if curr_line > max_width and ind > 0:
                split_string[ind - 1] += '\n'
                curr_line = 0
        return ''.join(split_string)
    
    # Shows the Calcudoku
    def show(self) -> None:
        canvas = Canvas(width=self.DISPLAY_SQUARE_WIDTH * self.n,
                        height=self.DISPLAY_SQUARE_HEIGHT * self.n)

        # draw the grid
        canvas.line_width = 0.5
    
        for i in range(self.n):
            for j in range(self.n):
                
                # Draw the squares
                p1 = (i*self.DISPLAY_SQUARE_HEIGHT, j*self.DISPLAY_SQUARE_WIDTH)
                canvas.stroke_rect(*p1, self.DISPLAY_SQUARE_HEIGHT, self.DISPLAY_SQUARE_WIDTH)
                
                # Calculate the first line of solution text
                canvas.font = self.SOLUTION_FONT
                t1 = self._add_tuples(p1, self.SOLUTION_OFFSET)
                
                # Split the text into lines
                solution_text = self._break_at(str(self.possible_values[i][j]), self.MAX_TEXT_WIDTH).split('\n')
                
                # Draw text with each line below the previous:
                for line_num, line in enumerate(solution_text):
                    line_offset = (0, self.LINE_BREAK_WIDTH * line_num)
                    canvas.fill_text(line, *self._add_tuples(t1, line_offset))
                
        
        # outline grid dark
        canvas.line_width = 5
        canvas.stroke_rect(0,0,self.DISPLAY_SQUARE_WIDTH * self.n, self.DISPLAY_SQUARE_HEIGHT * self.n)

        # draw the group boundaries
        canvas.line_width = 2.5
        for calc_square in self.calc_squares:
            dark_lines = self._calculate_dark_lines(calc_square)
            for line in dark_lines:
                self._draw_line(canvas, *line)
            
        # draw the group labels
        calculation_mapping = {
            multiply: '*',
            add: '+',
            divide: '/',
            subtract: '-',
            None: ''
        }
        canvas.font = self.GROUP_LABEL_FONT
        for calc_square in self.calc_squares:
            group_label_pos = self._grid_to_coords(*self._top_left_most(calc_square['positions']))
            group_label_pos = self._add_tuples(group_label_pos, self.GROUP_LABEL_OFFSET)
            group_label_text = str(calc_square['result']) + ' ' + calculation_mapping[calc_square['operation']]
            canvas.fill_text(group_label_text, *group_label_pos)
        
        display(canvas)

Not all Calcudokus are solvable though, so we can't randomly generate them without first writing an algorithm that guarentees solvability without guessing. Lucky for us the website: http://www.menneske.no/ already does this so for the purposes of this project we'll just write code that let's us pull Calcudokus from that website.

In [35]:
import requests
import re
import pickle
from typing import Tuple, List, Dict, Set
from bs4 import BeautifulSoup

def parse_visual_puzzle(visual_puzzle: Dict[str, object]):
    
    # A function to determine if a visual puzzle has a wall 
    # between visual positions a and visual positions b
    
    def is_walled(a: Dict[str, object], b: Dict[str, object]) -> bool:
        a_pos, b_pos = a['position'], b['position']
        if a_pos[0] == b_pos[0] and a_pos != b_pos:
            top = a if a_pos[1] < b_pos[1] else b
            return 'bottom' in top['borders']
        elif a_pos[1] == b_pos[1] and a_pos != b_pos:
            left = a if a_pos[0] < b_pos[0] else b
            return 'right' in left['borders']
        else:
            raise ValueError(f'Positions not adjacent {a_pos}, {b_pos}')
    
    # A recursive function that returns which squares are accesible from 
    # curr without crossing borders
    
    def reachable_squares(visual_puzzle: Dict[str, object], curr: Dict[str, object],
                          seen: Set[Tuple[int, int]]) -> List[Tuple[int, int]]:
        
        # You've seen the current square
        if not seen:
            seen = {curr['position'],}
        else:
            seen.add(curr['position'])
        
        offsets = [(0, 1), (1, 0), (-1, 0), (0, -1)]
        
        for offset in offsets:
            # Add the offset to curr
            offset_location = tuple(map(sum, zip(offset, curr['position'])))
            # Check if it exists
            exists = 0 <= offset_location[0] < n and 0 <= offset_location[1] < n
            
            
            # Add to seen all reachable squares from offset_square if there's no wall between them
            if exists and offset_location not in seen:
                # Flatten the puzzle
                flat_puzzle = [item for row in visual_puzzle for item in row]
            
                # Get the puzzle information at offset_location
                offset_info = list(filter(lambda a: a['position'] == offset_location, flat_puzzle))[0]

                walled = is_walled(curr, offset_info)
                if not walled:
                    reachable = reachable_squares(visual_puzzle, offset_info, seen)
                    for pos in reachable:
                        seen.add(pos)
                
        return seen
    
    n = len(visual_puzzle)
    
    # Roots are all cells with a result in them
    roots = [cell_info for row in visual_puzzle for cell_info in row if cell_info['result']]
    
    calc_groupings = []
    
    # A table mapping operation symbols to their functional counterparts
    calculation_mapping = {
        '*': multiply,
        '+': add,
        '-': subtract,
        '/': divide
    }
    
    # From the root, recursively spread outward to include all squares you can
    for root in roots:
        accessible = reachable_squares(visual_puzzle, root, None)
        group_info = {
            'result': int(root['result']),
            'positions': list(accessible)
        }
        
        # Parse and add the operation
        if root['operation']:
            group_info['operation'] = calculation_mapping[root['operation']]
        else:
            group_info['operation'] = None

        calc_groupings.append(group_info)

    return calc_groupings

def get_puzzle(size):    
    # For some reason a puzzle of 6 has a different url than any other
    if size != 6:
        request_url = f'http://www.menneske.no/calcudoku/{size}/eng/'
    else:
        request_url = f'http://www.menneske.no/calcudoku/eng/'
    
    response = requests.get(request_url)
    
    # Raise an exception if the request failed
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

        
    # Find the puzzle table
    table_body = soup.find('div', class_='grid2').find('table')
    
    # Get the table rows
    if table_body.find('tbody'): # If necessary, get what's inside the tbody tag
        table_body = table_body.find('tbody')
    rows = table_body.find_all('tr', recursive=False)

    # Parse the table row html to border & calculation information
    visual_puzzle = []
    for j, row in enumerate(rows):
        visual_puzzle_row = []
        cols = row.find_all('td', recursive=False)
        for i, col in enumerate(cols):
            # A mapping used to put border info in a more workable form
            border_mapping = {
                'normal': [],
                'bottomright': ['bottom', 'right'],
                'bottomedge': ['bottom'],
                'rightedge': ['right']
            }
            
            # Store the border on the current cell
            cell_info = {
                'position': (i,j),
                'borders': border_mapping[col['class'][0]]
            }
            
            # Look for the contents inside the cell
            cell_contents = col.find('table')
            if cell_contents.find('tbody'):
                cell_contents = cell_contents.find('tbody')
                
            # Parse the contents inside the inner-table
            math_box = cell_contents.find('td', class_='math')
            
            # Use regex to parse math_box or set 'result' and 'operation' to None
            if math_box.string:
                result_op_regex = r'^([0-9]+)([\+\-\*\/]*)$'
                regex_result = re.match(result_op_regex, math_box.string)
                cell_info['result'] = regex_result.group(1)
                if regex_result.group(2):
                    cell_info['operation'] = regex_result.group(2)
                else:
                    cell_info['operation'] = None                
            else:            
                cell_info['result'] = None
                cell_info['operation'] = None
            
            visual_puzzle_row.append(cell_info)
        visual_puzzle.append(visual_puzzle_row)
    return Board(size, parse_visual_puzzle(visual_puzzle))

get_puzzle(4).show()

Canvas(height=400, width=400)

Now let's grab a bunch of them and cache them for later so we don't have to deal with network delays when we start analyzing a large number of puzzles later on.

In [36]:
import os

class CacheManager:    
    CACHE_FILE_FORMAT_STRING = './cache/puzzle/{}.pickle'
    
    @staticmethod
    def read(n):
        file_path = CacheManager.CACHE_FILE_FORMAT_STRING.format(n)
        
        data = None
        if os.path.exists(file_path):
            try:
                with open(file_path, 'rb') as f:
                    data = pickle.load(f)
            except EOFError:
                pass
        return data


    @staticmethod
    def write(n, puzzles, append=True):
        file_path = CacheManager.CACHE_FILE_FORMAT_STRING.format(n)
        
        curr_file = CacheManager.read(n)
        
        if curr_file and append:
            curr_file = curr_file.extend(puzzles)
        else:
            curr_file = puzzles
        
        with open(file_path, 'wb') as f:
            pickle.dump(curr_file, f)
    
    @staticmethod
    def maximize_cache(puzzle_size, max_retry=7, limit=None):
        current_collection = CacheManager.read(puzzle_size)
        if current_collection is None:
            current_collection = []
        
        dupes_in_a_row = 0
        while dupes_in_a_row < max_retry and (not limit or len(current_collection) < limit):
            new_board = get_puzzle(puzzle_size)
            if new_board not in current_collection:
                current_collection.append(new_board)
                print(f'Found new board of size {puzzle_size}. Adding to collection...')
                print(f'Currently has {len(current_collection)} boards of size {puzzle_size}')
            else:
                dupes_in_a_row += 1
        
        CacheManager.write(puzzle_size, current_collection, append=False)
        print(f'Wrote {len(current_collection)} boards of size {puzzle_size} to cache')
    
    @staticmethod
    def backup_cache():
        pass
    
    @staticmethod
    def restore_cache(timestamp):
        pass
    
    @staticmethod
    def clear_cache():
        print('Clearing cache.')
        found = 0
        for i in range(4,10):
            file_path = CacheManager.CACHE_FILE_FORMAT_STRING.format(i)
            if os.path.exists(file_path):
                found += 1
                os.remove(file_path)
        print(f'Clear complete. Found {found} cached files')

## Designing The Algorithm

So the general plan for my algorithm would be to do an exhaustive search while using heuristics to limit the search space as much as possible.

### Time Complexity for (Super) Pure Exhaustive Search

Before we start using heuristics, let's try and get an upper limit of the time this algorithm would take it we just generated all possible squares, regardless of whether they were valid or not, and then checked them to see if they were the solution. 

Let n be the dimensions of the square, and m be the number of groups. Each square has $n$ possiblities and there are $n^2$ number of squares so that means that there are a total of $n^{n^2}$ number of possible arrangements. This is already prohibitively large number. For reference, the square used for the example above would have 4,294,967,296 possiblities. But let's keep calculating just for fun.

How long would it take to check whether each of the squares was valid? Well we would have to check every row for duplicates which we can do in $O(n^2)$ time, and every column which we can also do in $O(n^2)$ time. Then we'd have to check each group which we could do in $O(m)$ time. Since it's obvious that $m < n^2$ then we can check each square in $O(n^2)$ time.

Given that each square would take $O(n^2)$ time to check, and there are exactly $n^{n^2}$ squares, then our algorithm would run in $O(n^{n^2 + 2})$, which is litterally the worst run time I have ever seen. Let's see if we can do better (I sure hope we can).


### A Slightly More Reasonable Approach?

What if we use the fact that rows and columns can't contain the same number to limit our search space. There's no point generating a square with all 1's since we know that can't possibly be our answer. How many squares of this type are there?

Well it turns out this is a __really__ hard problem to solve. A square with no repeating number on either rows or columns is called a _Latin Square_. There's been significant research into calculating $L_n$, the number of Latin Squares with width and height n, but as of now no one has discovered a simple and effective formula. Not only that, but no one really knows how many Latin Squares are possible with any number greater than 11. But since our goal is to only solve Calcudokus with $n < 12$, let's check the values of $L_n$ up to 11.

<table border="1" cellpadding="5" cellspacing="0" align="center">
<caption>The numbers of Latin squares of various sizes
</caption>
<tbody><tr>
<th><span class="texhtml mvar" style="font-style:italic;">n</span></th>
<th align="right">all Latin squares of size <span class="texhtml mvar" style="font-style:italic;">n</span>
</th></tr>
<tr>
<td align="right">1</td>
<td align="right">1
</td></tr>
<tr>
<td align="right">2</td>
<td align="right">2
</td></tr>
<tr>
<td align="right">3</td>
<td align="right">12
</td></tr>
<tr>
<td align="right">4</td>
<td align="right">576
</td></tr>
<tr>
<td align="right">5</td>
<td align="right">161,280
</td></tr>
<tr>
<td align="right">6</td>
<td align="right">812,851,200
</td></tr>
<tr>
<td align="right">7</td>
<td align="right">61,479,419,904,000
</td></tr>
<tr>
<td align="right">8</td>
<td align="right">108,776,032,459,082,956,800
</td></tr>
<tr>
<td align="right">9</td>
<td align="right">5,524,751,496,156,892,842,531,225,600
</td></tr>
<tr>
<td>10</td>
<td align="right">9,982,437,658,213,039,871,725,064,756,920,320,000
</td></tr>
<tr>
<td>11</td>
<td align="right">776,966,836,171,770,144,107,444,346,734,230,682,311,065,600,000
</td></tr>
<tr>
<td>12
</td>
<td>?
</td></tr></tbody></table>

It seems that the numbers are workable up until about $n=6$, after which the numbers skyrocket. So that means that this approach is also computationally impossible.

### Limiting By Grouping

What if instead of looking at the square as a whole we looked at each individual grouping and limited our search space by which numbers can be in each group. For example, if $n=4$ and we had a group with three cells labeled `6+` we know that the values of those three cells have to be either `1+1+4`, `2+2+2`, or `3+1+2`. Additionally, we can eliminate `2+2+2` because there's no way to lay out three squares where the 2's would not either be in the same row or the same column. How would we do this computationally? Well an easy approach would be to: search all possible combinations of numbers in the squares, see which ones give you the desired result, then try all possible ways of laying them out within the grouping and seeing which ones are possible without having to repeat a number in the same row/column.

Let's do a quick, back of the envelope, complexity analysis for that solution to see if it's viable. Let's let $m$ be the number of groups, $w$ be the upper limit to the size of a group, and $n$ be the width/height of the Calcudoku. To search all possible combinations of numbers for a group we'd have to look through $n^w$ numbers. You might think that we would only have to look at $C\left(\frac{n}{w}\right)$ combinations but with certain operations, order matters. For example $1^3 \neq 3^1$ so checking the combination set `{1,3}` isn't enough you'd have to check `{1,3}` and `{3,1}`. As for checking all the ways to lay them out within a group, we'd have to generate $w!$ orderings for each valid ordering of numbers. Finally we'd have to check the validity of each order which we can do with a table and a single run through the values. So our final complexity would be:

$$T(n, w) \leq n^w * w! * w$$

Given that $n < 11$ and $w < 5$ for almost all cases we can get a rough estimate of the maximum number of operations:

$$T(11, 5) \leq 11^5 * 5! * 5 = 161051 * 120 * 5 = 96,630,600$$

That's pretty big but completely doable within a minute. Additionally if we lower the maximum size of $w$, we can see that the number shrinks dramatically.

$$T(11, 4) \leq 11^4 * 4! * 4 = 14641 * 24 * 4 = 1,405,536$$

So assuming that the size of the groups don't get too large then this strategy seems very viable. Let's write some code to do this:

In [37]:
from itertools import product, permutations
from typing import Tuple, List, Set

def filter_unique(p):
    p = map(sorted, p)
    seen = []
    p_unique = []
    for elem in p:
        if elem not in seen:
            p_unique.append(tuple(elem))
            seen.append(elem)
    return p_unique

def possible_group_arrangements(max_num, group_positions, operation, result, quiet=True):
    def validate_group(pos_val_mapping):
        x_check = set({})
        y_check = set({})
        for position, val in pos_val_mapping.items():
            if (position[0], val) in x_check or (position[1], val) in y_check:
                return False
            else:
                x_check.add((position[0], val))
                y_check.add((position[1], val))
        return True

    num_positions = len(group_positions)
    
    # A list of all num_positions-length orderings of numbers between [0,max_num]
    combos = list(product(range(1, max_num+1), repeat=num_positions))
    
    # Filter out the ones that don't result in the target when the operation is applied
    valid_combos = []
    for combo in combos:
        a = operation(*combo) 
        if a == result:
            valid_combos.append(combo)
    
    # Get rid of duplicate permutations of the same numbers
    valid_combos = filter_unique(valid_combos)

    # Zip together all the permutations of a valid ordering and group positions into a 
    # dictionary e.g.
    #
    # valid_combos = (1, 2)
    # group_positions = (0,0), (0,1)
    # 
    # yields 
    # 
    # [{(0,0): 1, (0,1): 2}, {(0,0): 2, (0,1): 1}]
    
    all_group_arrangements = []
        
    for valid_combo in valid_combos:
        combo_perms = list(permutations(valid_combo))
                
        for permutation in combo_perms:
            all_group_arrangements.append(dict(zip(group_positions, permutation)))

    d = list(filter(validate_group, all_group_arrangements))
    return list(filter(validate_group, all_group_arrangements))

def limit_possibilities_by_group_possibilities(board):
    for group in board.calc_squares:
        if group['operation']: 
            possible_arrangements = possible_group_arrangements(board.n,
                                                            group['positions'], group['operation'], group['result'])
            
            for individual_position in group['positions']:
                possible_individual_values = set({})
                for possible_arrangement in possible_arrangements:
                    possible_individual_values.add(possible_arrangement[individual_position])
                board[individual_position[1], individual_position[0]] = possible_individual_values
        else:
            for position in group['positions']:
                board[position[1], position[0]] = set([group['result']])

But how long does it actually take in practice? Good thing we have a handful of puzzles we can try it on!

In [38]:
import time

def test_function_on_puzzles(func, puzzles):
    times = []
    for puzzle in puzzles:
        start_time = time.time()
        func(puzzle)
        end_time = time.time()
        times.append(end_time - start_time)
    return times

What about time to complete as a function of size?

In [39]:
import matplotlib.pyplot as plt
from statistics import mean

def plot_func_time_per_size(timing_func):
    points = []

    for size in range(4,5):
        puzzles = CacheManager.read(size)
        times = test_function_on_puzzles(timing_func, puzzles)
        points.extend([(size, time) for time in times])

    plt.scatter(x=[x for x,y in points], y=[y for x,y in points])
    plt.show()

def plot_func_time_per_group_size(timing_func):
    def max_group_size():
        pass
    
    for size in range(4,9):
        puzzles = CacheManager.read(size)
        times = test_function_on_puzzles(timing_func, puzzles)
        group_size = list(map(max_group_size, puzzles))
        points.extend(list(zip(group, times)))

    plt.scatter(x=[x for x,y in points], y=[y for x,y in points])
    plt.show()

### Is it easy now?

Now that's we've shrunk our search space, can we just do an exhaustive search? This is going to be a little hard to find mathamatically, so I think the best way to find out is to try. Take, for example, the puzzle below:

In [42]:
from itertools import permutations
from numpy import array as arr
import time

# Takes as input a list of sets. Each set in the input list describes all possible values
# of the corresponding position of an element in the output list. The output list
# will contain all such possible lists. For example:
# 
# input = [{1, 2, 3}, {4, 5}, {6, 7}]
# output = [[1,3,4], [1,3,5], [2,3,4], [2,3,5]]

def line_permutations(list_of_possibilities, prev_rows=None):
#     if prev_rows:
#         zipped = zip(*prev_rows)
#         print(zipped)
#         for i, forbidden in enumerate(zipped):
#             print(forbidden)
#             list_of_possibilities[i] = list_of_possibilities[i].difference(set(forbidden))
    
    if len(list_of_possibilities) == 1:
        return list(map(lambda a: [a], list_of_possibilities[0]))

    all_perms = []
    for first_letter in list_of_possibilities[0]:
        k = [[y for y in x if y != first_letter] for x in list_of_possibilities[1:]]
        sub_possibilities = line_permutations(k)
        for sub_possibility in sub_possibilities:
            all_perms.append([first_letter, *sub_possibility])
    return all_perms


def all_possible_puzzles(puzzle, offset=0, prev_rows=None):
    if not prev_rows:
        prev_rows = []

    if len(prev_rows) == len(puzzle) - 1:
        return [[10 - a] for a in map(sum, zip(*matrix))]

    # get the puzzle row defined by offset
    row = puzzle[offset]

    # get the permutations only matching the current possible values
    # filter out the row permutations that conflict with the previous rows
    possible_rows = line_permutations(row, prev_rows)

    all_puzzles = []
    for possible_row in possible_rows:

        # get all possible combinations for the row below this one and append all possible
        # permutations of this row to it
        sub_puzzles = all_possible_puzzles(puzzle, offset + 1, [*prev_rows, possible_row])
        for sub_puzzle in sub_puzzles:
            all_puzzles.append([possible_row, *sub_puzzle])

    return all_puzzles

def validate_puzzle(puzzle, potential_answer):
    for calc_group in puzzle.calc_squares:
        if calc_group['operation']:
            values = [potential_answer[j][i] for i, j in calc_group['positions']]
            value_perms = permutations(values)
            solution = False
            for perm in value_perms:
                res = calc_group['operation'](*perm)
                if res == calc_group['result']:
                    solution = True
                    break

            if not solution:
                return False
        else:
            for pos in calc_group['positions']:
                if potential_answer[pos[1]][pos[0]] != calc_group['result']:
                    return False
    return True
    
def exhaustive_search(puzzle):
    limit_possibilities_by_group_possibilities(puzzle)
    puz_start = time.time()
    all_puz = all_possible_puzzles(puzzle)
    puz_end = time.time()
    sol_start = time.time()
    for solution in all_puz:
        if validate_puzzle(puzzle, solution):
            sol_end = time.time()
            print(f'Took {puz_end-puz_start} seconds to fetch all puzzles and {sol_end-sol_start} seconds to validate them.')
            return solution


In [ ]:
from random import randint
import matplotlib.pyplot as plt
import time
import pandas as pd
from itertools import permutations, product, combinations
from statistics import mean
import numpy as np
from math import factorial


class AlgAnalysis:
    def __init__(self, funcs, data, f_name='default', f_args=None, columns=None, force_reset=False):
        if columns is None:
            col_mapping = {}
        if f_args is None:
            f_args = []

        self.sample_size = len(data)
        self.file_path = f'./cache/analysis/{f_name}/{self.sample_size}.pickle'
        self.df = None
        cached = self._read_cache()
        if not force_reset and cached is not None:
            self.df = cached
        else:
            p = {'input': []}
            p.update({key: [] for key in columns.keys()})
            p.update({f.__name__: [] for f in funcs})
            count = 0

            for data_point in data:
                p['input'].append(data_point)
                for func in funcs:
                    p[func.__name__] = self._time_to_run(func, data, f_args=f_args)
                for col, col_func in columns.items():
                    p[col].append(col_func(data_point))
                count += 1
                print('Done with data point number ', count)
            self.df = DataFrame(p)
            self._write_cache()

    @staticmethod
    def _time_to_run(func, data, f_args=None):
        if f_args is None:
            f_args = []
        
        time_series = []
        for data_point in data:
            time_start = time.time()
            func(data_point, *f_args)
            time_end = time.time()
            time_series.append(time_end - time_start)
        return time_series

    def _write_cache(self):
        dir_name = os.path.dirname(self.file_path)
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
            
        with open(self.file_path, 'wb') as f:
            pickle.dump(self.df, f)
            print(f'Successfully cached {self.sample_size} results to {self.file_path}')
            
    def _read_cache(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, 'rb') as f:
                print(f'Reading {self.sample_size} results from {self.file_path}')
                return pickle.load(f)
        return None

    
    

def gen_data(number_points, width, vals, fullness):
    return [[set([randint(*vals) for _ in range(fullness)]) for _ in range(randint(*width))] for _ in range(number_points)]

    if columns is None:
        columns = {}
    if f_args is None:
        f_args = []
    
    cache_filepath = f'./line_permutation_testing_{size}_{f_name}.pickle'
    cached_file = CacheManager.read_data(cache_filepath)
    if force_reset or not cached_file:
        print('Generating new data.')
        d = gen_data(size, (4,9), (1,9), 9)
        print('Done generating data.')
        print('Testing functions:', *map(lambda a: a.__name__, funcs))
        alg = AlgAnalysis(funcs=funcs, data=d, col_mapping=columns, f_args=f_args)
        CacheManager.write_data(cache_filepath, alg)
        return alg
    else:
        print('Reading cached data.')
        return cached_file

    
    
def puzzle_data(sample_size=30, size_range=(4,)):
    data = []
    for _ in range(sample_size):
        data.append(get_puzzle(randint(*size_range)))
    return data

# 4 inital testing
# 5 filtered while creating permutations for rows
def test_exhaustive_search():
    data = CacheManager.read(4)
    data.extend(CacheManager.read(5))
    data.extend(CacheManager.read(6))
    res = AlgAnalysis([exhaustive_search], data, f_name='exhaustive_search_4', columns={
        'width': len,
    }, force_reset=True).df
    res['avg_group_size'] = res.apply(lambda b: mean(map(len, np.array(b['input']).flatten())), axis=1)
    res['max_group_size'] = res.apply(lambda b: max(map(len, np.array(b['input']).flatten())), axis=1)
    res = res.sort_values('avg_group_size')
    plt.scatter(res['max_group_size'], res['exhaustive_search'])
    plt.show()

test_exhaustive_search()

Reading 15 results from ./cache/analysis/exhaustive_search_4/15.pickle
Took 0.0001747608184814453 seconds to fetch all puzzles and 4.100799560546875e-05 seconds to validate them.
Took 0.02345418930053711 seconds to fetch all puzzles and 0.003042936325073242 seconds to validate them.
Took 0.0019152164459228516 seconds to fetch all puzzles and 0.0006158351898193359 seconds to validate them.
Took 0.0001990795135498047 seconds to fetch all puzzles and 0.0001938343048095703 seconds to validate them.
Took 0.028843164443969727 seconds to fetch all puzzles and 0.0031859874725341797 seconds to validate them.
Took 0.027366161346435547 seconds to fetch all puzzles and 0.0011909008026123047 seconds to validate them.
Took 0.300750732421875 seconds to fetch all puzzles and 0.03902316093444824 seconds to validate them.
Took 1.6505262851715088 seconds to fetch all puzzles and 0.5468509197235107 seconds to validate them.
Took 1.041031837463379 seconds to fetch all puzzles and 21.428325176239014 seconds